# Final Project for COGS118A, this is meant to find and identify russian 'troll' tweets.

Group members :
- Gael Van der Lee
- Alex Labranche

In [1]:
import sklearn as sk
import pandas as pd
import matplotlib.pyplot as plt
import time
import re
import random
from urllib.request import urlopen

# Data gathering and cleaning

### Loading and cleaning

In [2]:
# Load the 3 datasets
troll_users = pd.read_csv('Tweets/Trolls/users.csv')
troll_tweets = pd.read_csv('Tweets/Trolls/tweets.csv')
normal_tweets = pd.read_csv('Tweets/Normal/dashboard_x_usa_x_filter_nativeretweets.csv')

In [3]:
print('Columns of each dataframe at import :\n\nTroll users : {}\n\nTroll tweets : {}\n\nNormal tweets : {}'
      .format(troll_users.columns.values, troll_tweets.columns.values, normal_tweets.columns.values))

Columns of each dataframe at import :

Troll users : ['id' 'location' 'name' 'followers_count' 'statuses_count' 'time_zone'
 'verified' 'lang' 'screen_name' 'description' 'created_at'
 'favourites_count' 'friends_count' 'listed_count']

Troll tweets : ['user_id' 'user_key' 'created_at' 'created_str' 'retweet_count'
 'retweeted' 'favorite_count' 'text' 'tweet_id' 'source' 'hashtags'
 'expanded_urls' 'posted' 'mentions' 'retweeted_status_id'
 'in_reply_to_status_id']

Normal tweets : ['Tweet Id' 'Date' 'Hour' 'User Name' 'Nickname' 'Bio' 'Tweet content'
 'Favs' 'RTs' 'Latitude' 'Longitude' 'Country' 'Place (as appears on Bio)'
 'Profile picture' 'Followers' 'Following' 'Listed'
 'Tweet language (ISO 639-1)' 'Tweet Url']


In [4]:
# Make Nicknames consistent
troll_users['screen_name'] = troll_users['screen_name'].str.lower()

# Drop the irrelevant data
users_to_drop = ['statuses_count', 'time_zone', 'verified', 'favourites_count']
tweets_to_drop = ['created_at', 'tweet_id', 'source', 'posted', 'retweeted_status_id', 'retweeted', 'in_reply_to_status_id']
normal_to_drop = ['Tweet Id', 'Latitude', 'Longitude', 'Country', 'Profile picture', 'Tweet Url']
troll_users = troll_users.drop(users_to_drop, axis=1)
troll_tweets = troll_tweets.drop(tweets_to_drop, axis=1)
normal_tweets = normal_tweets.drop(normal_to_drop, axis=1)

In [5]:
users_new_cols = {'id': 'User Id', 'location': 'Location', 'name': 'User Name', 
                  'followers_count': 'Followers', 'lang': 'Language', 'screen_name': 'Nickname',
                  'description': 'Bio', 'created_at': 'Account creation date', 
                  'friends_count': 'Following', 'listed_count': 'Listed'}
tweets_new_cols = {'user_id': 'User Id', 'user_key': 'Nickname', 'created_str': 'Tweet date',
                   'retweet_count': 'Retweets', 'favorite_count': 'Favorites', 'text': 'Tweet',
                   'hashtags': 'Hashtags', 'expanded_urls': 'URLs', 'mentions': 'Mentions'}
normal_new_cols = {'Tweet content': 'Tweet', 'Favs': 'Favorites', 'RTs': 'Retweets', 
                   'Place (as appears on Bio)': 'Location', 'Tweet language (ISO 639-1)': 'Language'}

troll_users = troll_users.rename(columns=users_new_cols)
troll_tweets = troll_tweets.rename(columns=tweets_new_cols)
normal_tweets = normal_tweets.rename(columns=normal_new_cols)

# Make times formats the same
normal_tweets['Tweet date'] = normal_tweets['Date'] + ' ' + normal_tweets['Hour']
normal_tweets = normal_tweets.drop(['Date', 'Hour'], axis=1)

# Helps with merging
troll_users = troll_users.drop(['User Id', 'Account creation date'], axis=1)
troll_tweets = troll_tweets.drop('User Id', axis=1)

In [6]:
print('Columns of each dataframe after cleaning :\n\nTroll users : {}\n\nTroll tweets : {}\n\nNormal tweets : {}'
      .format(troll_users.columns.values, troll_tweets.columns.values, normal_tweets.columns.values))

Columns of each dataframe after cleaning :

Troll users : ['Location' 'User Name' 'Followers' 'Language' 'Nickname' 'Bio'
 'Following' 'Listed']

Troll tweets : ['Nickname' 'Tweet date' 'Retweets' 'Favorites' 'Tweet' 'Hashtags' 'URLs'
 'Mentions']

Normal tweets : ['User Name' 'Nickname' 'Bio' 'Tweet' 'Favorites' 'Retweets' 'Location'
 'Followers' 'Following' 'Listed' 'Language' 'Tweet date']


### Labeling and merging the datasets

In [7]:
# Merge the troll datasets
trolls = pd.merge(left=troll_tweets, right=troll_users)

In [8]:
trolls.iloc[:2, :]

,Nickname,Tweet date,Retweets,Favorites,Tweet,Hashtags,URLs,Mentions,Location,User Name,Followers,Language,Bio,Following,Listed
0,kathiemrr,2017-02-27 14:54:00,NaN,NaN,#ThingsDoneByMistake kissing auntie in the lips,"[""ThingsDoneByMistake""]",[],[],Atlanta,Kathie,2970.0,en,"Imperfection is beauty, madness is genius and ...",3157.0,22.0
1,kathiemrr,2016-11-28 16:17:36,NaN,NaN,RT @jadedsweetangel: #ToDoListBeforeChristmas ...,"[""ToDoListBeforeChristmas""]",[],[],Atlanta,Kathie,2970.0,en,"Imperfection is beauty, madness is genius and ...",3157.0,22.0


In [9]:
normal_tweets.iloc[:2, :]

,User Name,Nickname,Bio,Tweet,Favorites,Retweets,Location,Followers,Following,Listed,Language,Tweet date
0,Bill Schulhoff,BillSchulhoff,"Husband,Dad,GrandDad,Ordained Minister, Umpire...","Wind 3.2 mph NNE. Barometer 30.20 in, Rising s...",NaN,NaN,"East Patchogue, NY",386.0,705.0,24.0,en,2016-04-16 12:44
1,Daniele Polis,danipolis,"Viagens, geek, moda, batons laranja, cabelos c...",Pausa pro café antes de embarcar no próximo vô...,NaN,NaN,"Grapevine, TX",812.0,647.0,16.0,pt,2016-04-16 12:44


In [10]:
# Put labels
trolls['y'] = pd.Series([1 for x in range(len(trolls))], index=trolls.index)
normal_tweets['y'] = pd.Series([0 for x in range(len(normal_tweets))], index=normal_tweets.index)

In [11]:
# Merge normal and troll data 
tweets = trolls.append(normal_tweets, ignore_index=True)
print(tweets.shape)

(408271, 16)


In [12]:
# Random rows we will use for visualization
rows = [random.randint(0, len(tweets)) for i in range(4)]

In [13]:
tweets.iloc[rows, :]

,Bio,Favorites,Followers,Following,Hashtags,Language,Listed,Location,Mentions,Nickname,Retweets,Tweet,Tweet date,URLs,User Name,y
77808,"if you don't work hard to achieve your dreams,...",NaN,24344.0,21953.0,"[""IdRunForPresidentIf""]",en,113.0,"Pittsburgh, US",[],giselleevns,NaN,RT @HapennyHoHum: #IdRunForPresidentIf I could...,2016-12-07 15:20:57,[],Giselle Evans,1
261291,Malandrin Nation,NaN,437.0,156.0,NaN,en,5.0,"Hanford, CA",NaN,Malandrines,NaN,Making a pizza @ Fatte Alberts Pizza Co. https...,2016-04-16 00:14,NaN,Los Malandrines,0
286121,Follow this account for geo-targeted Cosmetolo...,NaN,235.0,225.0,NaN,en,9.0,"Chester Heights, PA",NaN,tmj_phl_cosmo,NaN,"#Cosmetology #Job in #GlenMills, PA: Hair Styl...",2016-04-15 20:21,NaN,TMJ-PHL Cosmo Jobs,0
322678,Seven continent marathoner (55 and counting). ...,NaN,423.0,582.0,NaN,en,9.0,"Florida, USA",NaN,paulbkny,NaN,Hanging out with my friend Bruce. #sharks #fam...,2016-04-15 15:44,NaN,Paul Nelson,0


### Extracting hashtags and mentions

In [14]:
def extract_hashtag(string):
    htags = []
    string = str(string)
    for word in string.split():
        if word[0] == '#':
            htags.append(word[1:])
    splits= []
    # Puts space before capital letters and numbers
    for h in htags:
        splits.append(re.sub(r"([A-Z0-9])", r" \1", h)[1:])
    return splits

def extract_mentions(string):
    men = []
    string = str(string)
    for word in string.split():
        if word[0] == '@':
            if word[-1] == ':':
                men.append(word[1:-1])
            else:
                men.append(word[1:])
    return men

# Not implemented yet, takes way too long to run on ~400,000 tweets
def extract_links(string):
    links = []
    string = str(string)
    for word in string.split():
        if word[:4] == 'http':
            try:
                link = urlopen(word).geturl()
                links.append(link)
            except:
                pass
    return links

def extract_website(string):
    string = str(string)
    start = string.find('//') + 2
    if start > 1:
        end = string[start:].find('/') + start
        return string[start:end]
    else:
        return ''

In [15]:
test = tweets['URLs'].apply(extract_website)
test[test != '']


330                  twitter.com
655              www.newsmax.com
696                wikileaks.org
1306                 twitter.com
1504                 twitter.com
1574                    paper.li
1621                     vine.co
2002                 twitter.com
2833                 twitter.com
3340                 twitter.com
3461                 twitter.com
3758                       ln.is
3874                       ln.is
4028            proudemocrat.com
4095                 twitter.com
4675                 twitter.com
5172           cards.twitter.com
5204                   on.cc.com
5634                 twitter.com
5719                 twitter.com
7333                 twitter.com
7337             blacktolive.org
7342                      bit.ly
7343                      bit.ly
7344                 twitter.com
7345                    youtu.be
7351                      bit.ly
7353                      bit.ly
7356                 twitter.com
7357           www.lifezette.com
          

In [16]:
trolls[(trolls['test'].str.len() != 0)][['Tweet', 'URLs', 'test']]

KeyError: 'test'

In [ ]:
tweets['Hashtags'] = tweets['Tweet'].apply(extract_hashtag)
tweets['Mentions'] = tweets['Tweet'].apply(extract_mentions)

In [ ]:
def clean(string):
    string = str(string)
    string = string.split()
    to_remove = []
    for word in string:
        if word[0] == '#' or word[0] == '@' or word == 'RT':
            to_remove.append(word)
    for word in to_remove:
        string.remove(word)
    return ' '.join(string)

In [ ]:
tweets['Tweet'] = tweets['Tweet'].apply(clean)

In [ ]:
tweets.iloc[rows, :]

# Different methods for NLP : One hot encoding and word2vec

In [ ]:
create 1 hot dataframe :
https://stackoverflow.com/questions/18889588/create-dummies-from-column-with-multiple-values-in-pandas

In [ ]:
Create word2vec dataframe

https://datascience.stackexchange.com/questions/10695/how-to-initialize-a-new-word2vec-model-with-pre-trained-model-weights
    

In [ ]:
Create training and validation set

# Different methods for classification : SVM and K Nearsest Neighbors 

NOTE: The gridsearch objects may require parameters to reduce memory consumption

In [ ]:
Create SVM
#initialize classifier objects for the 1-hot and word2vec encodings
svc_1_hot = svm.SVC(C=1.0, kernel='rbf', degree=3, gamma='auto', coef0=0.0, shrinking=True, probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False, max_iter=-1, decision_function_shape='ovr', random_state=None)

svc_word2vec = svm.SVC(C=1.0, kernel='rbf', degree=3, gamma='auto', coef0=0.0, shrinking=True, probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False, max_iter=-1, decision_function_shape='ovr', random_state=None)

#initialize parameters

#penalty parameter
C_list =[10**(1-i) for i in range(6)]
#kernel function - poly:polynomial
kernel_list =['linear','rbf','sigmoid','poly']
param_dic_SVM = {'C':C_list, 'kernel':kernel_list}

In [ ]:
Train SVM using GridSearch on one-hot
save results in list

clf_svm_1 = GridSearchCV(estimator, param_dict_SVM, cv=5, refit = True)


In [ ]:
Train SVM using Gridsearch on w2v
save results in list

clf_svm_2 = GridSearchCV(estimator, param_dict_SVM, cv=5, refit = True)


In [ ]:
Create KNN

knn_one_hot = neighbors.KNeighborsClassifier(n_jobs=-1)

knn_word2vec = neighbors.KNeighborsClassifier(n_jobs=-1)

#initialize parameters

#number of nearest neighbors to consider: 3-10
n_neighbors_list = [i+3 for i in range(8)]
#weighting of neighbors, whether they all get an equal vote or are weighted by distance
weights_list = ['uniform','distance']
param_dic_knn = {'n_neighbors':n_neighbors_list,'weights':weights_list}


In [ ]:
Train KNN using GridSearch on one-hot
save results in list

clf_knn_1 = GridSearchCV(knn_one_hot, param_dict_knn, cv=5, refit = True)

In [ ]:
Train KNN using Gridsearch on w2v
save results in list

clf_knn_2 = GridSearchCV(knn_word2vec, param_dict_knn, cv=5, refit = True)

# Results

In [ ]:
Plot and present results